## Cтроим базовый RAG pipeline

## Устанавливаем api ключи и прокси 

In [11]:
import os
os.environ["OPENAI_API_KEY"] = "sk-kFANI4tptQfVJAZKpbRxDNBVODFDrOdk"
os.environ["OPENAI_BASE_URL"] = 'https://api.proxyapi.ru/openai/v1'

## Импортируем нужные библиотеки

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Загружаем документы

In [13]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("/Applications/programming/kg_llm/assets/скан-архив/install_and_settings.docx")
docs = loader.load()

## Разбиваем документы на чанки

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

## Индексируем наши чанки 

In [21]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

## Загружаем LLM

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(base_url='https://api.proxyapi.ru/openai/v1')

## Строим prompt-инструкцию для LLM

In [23]:
prompt = ChatPromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Question: {question} 
Context: {context} 
Answer:""")

## Собираем RAG-CHAIN
` {"context": retriever | format_docs, "question": RunnablePassthrough()}`


`"context": retriever | format_docs` - контекст строится по цепочке 
1. `retriever` возвращает нам близкие чанки. 
2. `format_docs` форматирует текст чанков.

`"question": RunnablePassthrough()` - позволяет нам передать вопрос от пользователя в цепочку `retriever | format_docs` для получения контекста 


Таким обрзом при вызове  RAG цепочки
1. формируется словарь: `{"context": "context from retriver", "question": "user question"}`
2. словарь отпраляется в `prompt`.
3. `prompt` подается в LLM.
4.  LLM возвращает ответ и ` StrOutputParser()` просто отдает его назад без обработки. 




In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
rag_chain.invoke("Как продлить активацию, как выполнить активацию")

'Для продления активации необходимо приобрести новую лицензию и выполнить активацию этой лицензии на рабочем месте пользователя. Активацию новой лицензии можно выполнить до истечения срока действия предыдущей лицензии.'